In [308]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

from crashpy.dataclasses import spectrum 
import scipy.constants as constants

In [ ]:
column_names=['x', 'y', 'z', 'a', 'Q', 'b', 'name', 'c',
'halo', 'd', 'packets', 'e', 'loc_uv', 'f', 'loc_x']
sources_path = '/ptmp/mpa/mglatzle/TNG_f_esc/e6/input/L/sn008/g111/sources_ic00.in'
sources = pd.read_table(sources_path, delimiter='\t', names=column_names)
sources['Q'] = sources['Q'].str.replace('d','e').astype('float')
sources_updated = sources[sources['Q']>0.01*sources['Q'].max()]
sources_updated['Q'] = sources_updated['Q'].astype(str)
sources_updated.to_csv(sources_path, sep='\t', header=False, index=False)

In [223]:
column_names=['x', 'y', 'z', 'a', 'Q', 'b', 'name', 'c',
'halo', 'd', 'packets', 'e', 'loc_uv', 'f', 'loc_x']
sources_path = '/ptmp/mpa/mglatzle/TNG_f_esc/merger_testing/input/L/sn008/g1029/sources_ic00.in'
sources = pd.read_table(sources_path, delimiter='\t', names=column_names)
#sources['Q'] = sources['Q'].str.replace('d','e').astype('float')
#sources['Q'] = sources['Q'].str.replace('d','e').astype('float')

In [193]:
lum_sources = sources[sources['Q']>0.01*sources['Q'].max()]
faint_sources = sources[sources['Q']<0.01*sources['Q'].max()]

In [309]:
def get_nearest_lum_source(faint_source, lum_sources):
    distances = []
    for _, source in lum_sources.iterrows():
        distance = np.sqrt((source.x-faint_source.x)**2+
                          (source.y-faint_source.y)**2+
                          (source.z-faint_source.z)**2)
        distances.append(distance)
    distances = np.array(distances)
    min_index = np.argmin(distances)
    min_value = distances[min_index]
    index = lum_sources.iloc[min_index].name
    return min_value, index

In [310]:
def source_merger(source_df, lum_source_idx, faint_source_idx, path_to_halo_spec):
    faint_source = source_df.loc[faint_source_idx]
    lum_source = source_df.loc[lum_source_idx]
    
    faint_spec_path = os.path.join(path_to_halo_spec, faint_source['name'][1:-1]+'.ps')
    lum_spec_path = os.path.join(path_to_halo_spec, lum_source['name'][1:-1]+'.ps')
    
    spec_faint = spectrum.CrashSpectrum.fromFile(faint_spec_path)
    spec_lum = spectrum.CrashSpectrum.fromFile(lum_spec_path)
    
    lum_faint = faint_source['Q']
    lum_lum = lum_source['Q']
    
    spec_faint.setEmissivity(lum_faint)
    spec_lum.setEmissivity(lum_lum)
    
    summed_spectrum = spectrum.CrashSpectrum(constants.e/constants.h*spec_lum.spectrum['hnu'], 
                                             spec_lum.spectrum['Lnu']+spec_faint.spectrum['Lnu'])
    
    out_path = lum_spec_path
    #out_path = os.path.join(path_to_halo_spec, f'test{lum_source_idx}.ps')
    summed_spectrum.toFile(path=out_path, f=True)
    
    source_df.loc[lum_source_idx,'Q'] = lum_faint+lum_lum
    source_df.drop(faint_source_idx, inplace=True)
    return

In [311]:
def merge_halo_sources(path_to_input, path_to_spectra, snap, halo_id, threshold=0.01, max_dist=1.9):
    column_names=['x', 'y', 'z', 'a', 'Q', 'b', 'name', 'c',
    'halo', 'd', 'packets', 'e', 'loc_uv', 'f', 'loc_x']
    sources_path = os.path.join(path_to_input, snap, f'g{halo_id}','sources_ic00.in')
    spectra_path = os.path.join(path_to_spectra, snap, f'g{halo_id}')
    
    source_df = pd.read_table(sources_path, delimiter='\t', names=column_names)
    source_df['Q'] = source_df['Q'].str.replace('d','e').astype('float')
    
    lum_sources = source_df[source_df['Q']>threshold*source_df['Q'].max()]
    faint_sources = source_df[source_df['Q']<threshold*source_df['Q'].max()]
    for faint_idx, source in faint_sources.iterrows():
        dist, lum_source_idx = get_nearest_lum_source(faint_source=source, lum_sources=lum_sources)
        if dist > max_dist:
            source_df.drop(faint_idx, inplace=True)
        else:
            source_merger(source_df=source_df, lum_source_idx=lum_source_idx, 
                          faint_source_idx=faint_idx, 
                          path_to_halo_spec=spectra_path)
            
    source_df['Q'] = source_df['Q'].apply('{:.3e}'.format)
    source_df['Q'] = source_df['Q'].str.replace('e','d')
    source_df.to_csv(sources_path, sep='\t', header=False, index=False)
    return

In [313]:
def get_all_halos(path, snap):
    path_to_snap = os.path.join(path, snap)
    files = os.listdir(path_to_snap)
    halos = [int(halo[1:]) for halo in files if halo.startswith('g')]
    return halos

In [317]:
def all_halo_merge(path_to_input, path_to_spectra, threshold=0.01, max_dist=1.9):
    snaps = ['sn004','sn008','sn013']
    for snap in snaps:
        halos = get_all_halos(path_to_input, snap)
        for halo in halos:
            print(f'Merging sources in halo {halo} in snap {snap}')
            merge_halo_sources(path_to_input=path_to_input, path_to_spectra=path_to_spectra, snap=snap, 
                               halo_id=halo, threshold=threshold, max_dist=max_dist)
    return

In [326]:
path_to_input = '/ptmp/mpa/mglatzle/TNG_f_esc/source_conv/input/L35n2160TNG'
path_to_spectra = '/ptmp/mpa/mglatzle/TNG_f_esc/source_conv/db/SPECTRA/L35n2160TNG'
snap='sn008'
merge_halo_sources(path_to_input, path_to_spectra, snap, halo_id=1, threshold=0.001, max_dist=1.9)
merge_halo_sources(path_to_input, path_to_spectra, snap, halo_id=100, threshold=0.005, max_dist=1.9)
merge_halo_sources(path_to_input, path_to_spectra, snap, halo_id=111, threshold=0.01, max_dist=1.9)
merge_halo_sources(path_to_input, path_to_spectra, snap, halo_id=1029, threshold=0.05, max_dist=1.9)

In [324]:
path_to_input = '/ptmp/mpa/mglatzle/TNG_f_esc/conv1e9/input/L35n2160TNG'
path_to_spectra = '/ptmp/mpa/mglatzle/TNG_f_esc/conv1e9/db/SPECTRA/L35n2160TNG'
all_halo_merge(path_to_input, path_to_spectra, threshold=0.01, max_dist=1.9)

Merging sources in halo 7348 in snap sn004
Merging sources in halo 6053 in snap sn004
Merging sources in halo 758 in snap sn004
Merging sources in halo 218 in snap sn004
Merging sources in halo 825 in snap sn004
Merging sources in halo 872 in snap sn004
Merging sources in halo 2048 in snap sn004
Merging sources in halo 214 in snap sn004
Merging sources in halo 3995 in snap sn004
Merging sources in halo 2184 in snap sn004
Merging sources in halo 2302 in snap sn004
Merging sources in halo 71 in snap sn004
Merging sources in halo 663 in snap sn004
Merging sources in halo 611 in snap sn004
Merging sources in halo 3854 in snap sn004
Merging sources in halo 1415 in snap sn004
Merging sources in halo 361 in snap sn004
Merging sources in halo 4276 in snap sn004
Merging sources in halo 4679 in snap sn008
Merging sources in halo 2634 in snap sn008
Merging sources in halo 5604 in snap sn008
Merging sources in halo 791 in snap sn008
Merging sources in halo 386 in snap sn008
Merging sources in halo

In [307]:
path_to_input = '/ptmp/mpa/mglatzle/TNG_f_esc/merger8/input/L'
path_to_spectra = '/ptmp/mpa/mglatzle/TNG_f_esc/merger8/db/SPECTRA/L35n2160TNG'
snap = 'sn008'
halo_id = 1029
merge_halo_sources(path_to_input, path_to_spectra, snap, halo_id, threshold=0.01, max_dist=5.3)

In [322]:
def get_job_list(jobnum=200):
    jobs = ''
    folder_name = os.getcwd().split('/')[-1]
    job_name = f'run-{folder_name}-chunk'
    for i in range(jobnum):
        jobs += f'{job_name}{i},'
    return jobs

In [320]:
jobs

'run-analysis-chunk0,run-analysis-chunk1,run-analysis-chunk2,run-analysis-chunk3,run-analysis-chunk4,run-analysis-chunk5,run-analysis-chunk6,run-analysis-chunk7,run-analysis-chunk8,run-analysis-chunk9,run-analysis-chunk10,run-analysis-chunk11,run-analysis-chunk12,run-analysis-chunk13,run-analysis-chunk14,run-analysis-chunk15,run-analysis-chunk16,run-analysis-chunk17,run-analysis-chunk18,run-analysis-chunk19,run-analysis-chunk20,run-analysis-chunk21,run-analysis-chunk22,run-analysis-chunk23,run-analysis-chunk24,run-analysis-chunk25,run-analysis-chunk26,run-analysis-chunk27,run-analysis-chunk28,run-analysis-chunk29,run-analysis-chunk30,run-analysis-chunk31,run-analysis-chunk32,run-analysis-chunk33,run-analysis-chunk34,run-analysis-chunk35,run-analysis-chunk36,run-analysis-chunk37,run-analysis-chunk38,run-analysis-chunk39,run-analysis-chunk40,run-analysis-chunk41,run-analysis-chunk42,run-analysis-chunk43,run-analysis-chunk44,run-analysis-chunk45,run-analysis-chunk46,run-analysis-chunk47,r

In [195]:
path_to_halo_spec = '/ptmp/mpa/mglatzle/TNG_f_esc/merger_testing/db/SPECTRA/L35n2160TNG/sn008/g1029'
faint_source1 = sources.loc[2]
faint_source2 = sources.loc[4]
faint_source3 = sources.loc[23]
faint_source4 = sources.loc[41]

lum_source = sources.loc[1]

faint_spec_path1 = os.path.join(path_to_halo_spec, faint_source1['name'][1:-1]+'.ps')
faint_spec_path2 = os.path.join(path_to_halo_spec, faint_source2['name'][1:-1]+'.ps')
faint_spec_path3 = os.path.join(path_to_halo_spec, faint_source3['name'][1:-1]+'.ps')
faint_spec_path4 = os.path.join(path_to_halo_spec, faint_source4['name'][1:-1]+'.ps')

lum_spec_path = os.path.join(path_to_halo_spec, lum_source['name'][1:-1]+'.ps')

spec_faint1 = spectrum.CrashSpectrum.fromFile(faint_spec_path1)
spec_faint2 = spectrum.CrashSpectrum.fromFile(faint_spec_path2)
spec_faint3 = spectrum.CrashSpectrum.fromFile(faint_spec_path3)
spec_faint4 = spectrum.CrashSpectrum.fromFile(faint_spec_path4)
spec_lum = spectrum.CrashSpectrum.fromFile(lum_spec_path)

lum_faint1 = faint_source1['Q']
lum_faint2 = faint_source2['Q']
lum_faint3 = faint_source3['Q']
lum_faint4 = faint_source4['Q']
lum_lum = lum_source['Q']

spec_faint1.setEmissivity(lum_faint1)
spec_faint2.setEmissivity(lum_faint2)
spec_faint3.setEmissivity(lum_faint3)
spec_faint4.setEmissivity(lum_faint4)
spec_lum.setEmissivity(lum_lum)

summed_spectrum = spectrum.CrashSpectrum(constants.e/constants.h*spec_lum.spectrum['hnu'], 
                                         spec_lum.spectrum['Lnu']+spec_faint1.spectrum['Lnu']+spec_faint2.spectrum['Lnu']+
                                         +spec_faint3.spectrum['Lnu']+spec_faint4.spectrum['Lnu'])

out_path = os.path.join(path_to_halo_spec, f'test.ps')
summed_spectrum.toFile(path=out_path, f=True)